In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 40)
pd.set_option('display.float_format', '{:.2f}'.format)

In [2]:
df = pd.read_csv('dataApt.csv', encoding='1251')
df.head(2)

,DR_Dat,DR_Tim,DR_NChk,DR_NDoc,DR_Apt,DR_Kkm,DR_TDoc,DR_TPay,DR_CDrugs,DR_NDrugs,DR_Suppl,DR_Prod,DR_Kol,DR_CZak,DR_CRoz,DR_SDisc,DR_CDisc,DR_BCDisc,DR_TabEmpl,DR_VZak,DR_Pos
0,2022-08-11,10:15:35,2173,2004598,2,22577,Розничная реализация,18,45399,ЦИПРОЛЕТ 3МГ/МЛ. 5МЛ. №1 ГЛ.КАПЛИ ФЛ./КАП. /Д-...,Катрен г.Химки,Д-р Редди с Лабораторис Лтд / Dr.REDDY's,1.00,41.08,51.00,12.00,925.00,200000000492.00,205,1,1.00
1,2022-08-11,10:27:46,2174,2004598,2,22577,Розничная реализация,15,261519,ПЕРЕКИСЬ ВОДОРОДА 3% 100МЛ. №40 Р-Р ФЛ.,Катрен г.Химки,ФЛОРА КАВКАЗА ОАО,1.00,18.61,31.00,3.00,9.00,200010010204.00,205,1,1.00


**One-dimensional ABC-analysis:**

In [12]:
groupped_df = df.groupby('DR_NDrugs').agg({'DR_Kol': sum})
groupped_df = (groupped_df['DR_Kol'] / sum(groupped_df['DR_Kol'])).sort_values(ascending=False)
groupped_df = groupped_df.cumsum().reset_index()
groupped_df['abc'] = np.where(groupped_df['DR_Kol'] < 0.8, 'A', np.where(groupped_df['DR_Kol'] < 0.95, 'B', 'C'))
groupped_df

,DR_NDrugs,DR_Kol,abc
0,ПАКЕТ,0.03,A
1,"ЦЕФТРИАКСОН 1Г. №1 ПОР. Д/Р-РА Д/В/В,В/М ФЛ. /...",0.04,A
2,"НАФТИЗИН 0,1% 15МЛ. НАЗАЛ.КАПЛИ ФЛ./КАП. /ЛЕККО/",0.05,A
3,"ЛЕЙКОПЛАСТЫРЬ БАКТЕР. 2,5Х7,2 №1 /ВЕРОФАРМ/",0.06,A
4,"СНУП 0,1% 90МКГ/ДОЗА 15МЛ. НАЗАЛ.СПРЕЙ ФЛ. /ШТ...",0.07,A
...,...,...,...
1871,"НАТРИЯ ХЛОРИД 0,9% 400МЛ. №16 Р-Р Д/ИНФ. КОНТ....",1.00,C
1872,"СФМ ШПРИЦ 2МЛ. 3-Х КОМП. 0,63Х32ММ 23G №100 [SFM]",1.00,C
1873,"СФМ ШПРИЦ 50МЛ. 3-Х КОМП. 1,2X40ММ 18G №25 [SFM]",1.00,C
1874,ДЕРМАГРИП ХАЙ РИСК ПЕРЧАТКИ ЛАТ. СМОТР. Н/СТЕР...,1.00,C


**Multidimensional ABC-analysis:**

In [12]:
groupped_df = df.groupby('DR_NDrugs').agg({'DR_Kol': sum, 'DR_CRoz': sum})
groupped_df['rel_kol'] = groupped_df['DR_Kol'] / sum(groupped_df['DR_Kol'])
groupped_df = groupped_df.sort_values('rel_kol', ascending=False)
groupped_df['cumsum_kol'] = groupped_df['rel_kol'].cumsum()
groupped_df['abc_kol'] = np.where(groupped_df['cumsum_kol'] < 0.8, 'A', np.where(groupped_df['cumsum_kol'] < 0.95, 'B', 'C'))
groupped_df['rel_roz'] = groupped_df['DR_CRoz'] / sum(groupped_df['DR_CRoz'])
groupped_df = groupped_df.sort_values('rel_roz', ascending=False)
groupped_df['cumsum_roz'] = groupped_df['rel_roz'].cumsum()
groupped_df['abc_roz'] = np.where(groupped_df['cumsum_roz'] < 0.8, 'A', np.where(groupped_df['cumsum_roz'] < 0.95, 'B', 'C'))
groupped_df[['DR_CRoz', 'DR_Kol', 'abc_kol', 'abc_roz']].reset_index().sort_values('DR_CRoz', ascending=False)


,DR_NDrugs,DR_CRoz,DR_Kol,abc_kol,abc_roz
0,"ТЕРАФЛЮ ЛИМОН ОТ ГРИППА И ПРОСТУДЫ 22,1Г. №14 ...",15638.00,7.00,A,A
1,НИМЕСИЛ 100МГ. 2Г. №30 ГРАН. Д/СУСП. Д/ПРИЕМА ...,14445.00,2.80,A,A
2,ИНГАВИРИН 90МГ. №10 КАПС. /ВАЛЕНТА/,13750.00,18.00,A,A
3,ХАРТМАНН БРАНОЛИНД H ПОВЯЗКА СТЕР. 10Х20СМ. №3...,12908.00,0.17,C,A
4,"ЭЛИКВИС 2,5МГ. №60 ТАБ. П/П/О /ПФАЙЗЕР/БРИСТОЛ...",12731.00,5.00,A,A
...,...,...,...,...,...
1871,БАХИЛЫ №10 (5ПАР),10.00,1.00,A,C
1872,КОНТЕЙНЕР Д/СБОРА БИОМАТЕРИАЛА 60МЛ. +ШПАТЕЛЬ ...,10.00,1.00,B,C
1873,АСКОРБИНОВАЯ К-ТА 25МГ. №10 ТАБ. КРУТКА САХ. /...,9.00,7.00,A,C
1874,КЛИНСА БАХИЛЫ СТАНДАРТ №2 (1ПАРА),7.00,2.00,A,C


**Automated ABC-analysis:**

In [14]:
def perform_abc(df, index):
    cols = list(df.columns)
    cols.remove(index)
    groupped_df = df.groupby(index).agg({col: sum for col in cols})
    for col in cols:
        groupped_df[f'rel_{col}'] = groupped_df[col] / sum(groupped_df[col])
        groupped_df = groupped_df.sort_values(f'rel_{col}', ascending=False)
        groupped_df[f'cumsum_{col}'] = groupped_df[f'rel_{col}'].cumsum()
        groupped_df[f'abc_{col}'] = np.where(groupped_df[f'cumsum_{col}'] < 0.8, 'A', np.where(groupped_df[f'cumsum_{col}'] < 0.95, 'B', 'C'))
    return groupped_df[cols + [f'abc_{col}' for col in cols]]
        

In [13]:
df1 = df[['DR_NDrugs', 'DR_Kol', 'DR_CRoz']]
df1.columns

Index(['DR_NDrugs', 'DR_Kol', 'DR_CRoz'], dtype='object')

In [15]:
perform_abc(df1, index='DR_NDrugs')

,DR_Kol,DR_CRoz,abc_DR_Kol,abc_DR_CRoz
DR_NDrugs,,,,
"ТЕРАФЛЮ ЛИМОН ОТ ГРИППА И ПРОСТУДЫ 22,1Г. №14 ПОР. Д/Р-РА Д/ПРИЕМА ВНУТРЬ ПАК.",7.00,15638.00,A,A
НИМЕСИЛ 100МГ. 2Г. №30 ГРАН. Д/СУСП. Д/ПРИЕМА ВНУТРЬ ПАК. /ГУИДОТТИ/МЕНАРИНИ/,2.80,14445.00,A,A
ИНГАВИРИН 90МГ. №10 КАПС. /ВАЛЕНТА/,18.00,13750.00,A,A
ХАРТМАНН БРАНОЛИНД H ПОВЯЗКА СТЕР. 10Х20СМ. №30 ПЕРУАН.БАЛЬЗАМ /АРТ.4923462/ [BRANOLIND],0.17,12908.00,C,A
"ЭЛИКВИС 2,5МГ. №60 ТАБ. П/П/О /ПФАЙЗЕР/БРИСТОЛ-МАЙЕРС/",5.00,12731.00,A,A
...,...,...,...,...
БАХИЛЫ №10 (5ПАР),1.00,10.00,A,C
КОНТЕЙНЕР Д/СБОРА БИОМАТЕРИАЛА 60МЛ. +ШПАТЕЛЬ СТЕР. И/У (БАНКА),1.00,10.00,B,C
АСКОРБИНОВАЯ К-ТА 25МГ. №10 ТАБ. КРУТКА САХ. /АСКОПРОМ/,7.00,9.00,A,C


**Sales dynamics:**

In [17]:
df = pd.read_csv('dataApt.csv', encoding='1251')

In [18]:
df.head(2)

,DR_Dat,DR_Tim,DR_NChk,DR_NDoc,DR_Apt,DR_Kkm,DR_TDoc,DR_TPay,DR_CDrugs,DR_NDrugs,DR_Suppl,DR_Prod,DR_Kol,DR_CZak,DR_CRoz,DR_SDisc,DR_CDisc,DR_BCDisc,DR_TabEmpl,DR_VZak,DR_Pos
0,2022-08-11,10:15:35,2173,2004598,2,22577,Розничная реализация,18,45399,ЦИПРОЛЕТ 3МГ/МЛ. 5МЛ. №1 ГЛ.КАПЛИ ФЛ./КАП. /Д-...,Катрен г.Химки,Д-р Редди с Лабораторис Лтд / Dr.REDDY's,1.00,41.08,51.00,12.00,925.00,200000000492.00,205,1,1.00
1,2022-08-11,10:27:46,2174,2004598,2,22577,Розничная реализация,15,261519,ПЕРЕКИСЬ ВОДОРОДА 3% 100МЛ. №40 Р-Р ФЛ.,Катрен г.Химки,ФЛОРА КАВКАЗА ОАО,1.00,18.61,31.00,3.00,9.00,200010010204.00,205,1,1.00


In [ ]:
df.DR_Dat.unique()

array(['2022-08-01', '2022-08-02', '2022-08-03', '2022-08-04',
       '2022-08-05'], dtype=object)

In [ ]:
df.DR_Apt.unique()

array([13], dtype=int64)

In [ ]:
df1 = pd.read_csv('data/data1.csv', encoding='1251')
df1 = df1.groupby('DR_Dat').apply(lambda x: pd.Series({'revenue': sum(x['DR_Kol']*x['DR_CRoz'] - x['DR_SDisc'])})).reset_index()
df1

,DR_Dat,revenue
0,2022-08-01,84681.52
1,2022-08-02,71389.37
2,2022-08-03,78050.82
3,2022-08-04,59187.36
4,2022-08-05,56458.81


In [ ]:
df = df.groupby('DR_Dat').apply(lambda x: sum(x['DR_Kol']*x['DR_CRoz'] - x['DR_SDisc'])).reset_index()
df

,DR_Dat,0
0,2022-08-01,84681.52
1,2022-08-02,71389.37
2,2022-08-03,78050.82
3,2022-08-04,59187.36
4,2022-08-05,56458.81


In [ ]:
df.columns = ['DR_Dat', 'revenue']

In [ ]:
df

,DR_Dat,revenue
0,2022-08-01,84681.52
1,2022-08-02,71389.37
2,2022-08-03,78050.82
3,2022-08-04,59187.36
4,2022-08-05,56458.81


In [ ]:
df['revenue_d'] = df['revenue'].rolling(2).apply(lambda x: (x.iloc[1] - x.iloc[0])/x.iloc[0])
df

,DR_Dat,revenue,revenue_d
0,2022-08-01,84681.52,NaN
1,2022-08-02,71389.37,-0.16
2,2022-08-03,78050.82,0.09
3,2022-08-04,59187.36,-0.24
4,2022-08-05,56458.81,-0.05


In [ ]:
df['revenue_shifted'] = df['revenue'].shift(1)

In [ ]:
df

,DR_Dat,revenue,revenue_d,revenue_shifted
0,2022-08-01,84681.52,NaN,NaN
1,2022-08-02,71389.37,-0.16,84681.52
2,2022-08-03,78050.82,0.09,71389.37
3,2022-08-04,59187.36,-0.24,78050.82
4,2022-08-05,56458.81,-0.05,59187.36


In [ ]:
df['revenue_d2'] = (df['revenue'] - df['revenue_shifted']) / df['revenue_shifted']
df

,DR_Dat,revenue,revenue_d,revenue_shifted,revenue_d2
0,2022-08-01,84681.52,NaN,NaN,NaN
1,2022-08-02,71389.37,-0.16,84681.52,-0.16
2,2022-08-03,78050.82,0.09,71389.37,0.09
3,2022-08-04,59187.36,-0.24,78050.82,-0.24
4,2022-08-05,56458.81,-0.05,59187.36,-0.05


In [ ]:
df['revenue_d3'] = df['revenue'].pct_change(1)
df

,DR_Dat,revenue,revenue_d,revenue_shifted,revenue_d2,revenue_d3
0,2022-08-01,84681.52,NaN,NaN,NaN,NaN
1,2022-08-02,71389.37,-0.16,84681.52,-0.16,-0.16
2,2022-08-03,78050.82,0.09,71389.37,0.09,0.09
3,2022-08-04,59187.36,-0.24,78050.82,-0.24,-0.24
4,2022-08-05,56458.81,-0.05,59187.36,-0.05,-0.05


**XYZ-анализ**

In [ ]:
df = pd.read_csv('data/data1.csv', encoding='1251')

In [ ]:
df = df.head()

,DR_Dat,DR_Tim,DR_NChk,DR_NDoc,DR_Apt,DR_Kkm,DR_TDoc,DR_TPay,DR_CDrugs,DR_NDrugs,DR_Suppl,DR_Prod,DR_Kol,DR_CZak,DR_CRoz,DR_SDisc,DR_CDisc,DR_BCDisc,DR_TabEmpl,DR_VZak,DR_Pos
0,2022-08-01,08:06:18,1272,13002561,13,22589,Розничная реализация,18,144734,ГАСТАЛ №12 ТАБ. Д/РАСС.,Пульс,TEVA Pharvaceutical Industries Ltd,1.00,196.71,270.00,0.00,NaN,NaN,29,1,1.00
1,2022-08-01,08:38:53,1273,13002561,13,22589,Розничная реализация,15,69661,"ТОБРОПТ 0,3% 5МЛ. №1 ГЛ.КАПЛИ ФЛ./КАП. /РОМФАРМ/",Пульс,РОМФАРМ КОМПАНИ ( ROMPHARM ),1.00,106.21,127.00,6.00,9.00,200010004357.00,29,1,1.00
2,2022-08-01,08:55:38,1274,13002561,13,22589,Розничная реализация,18,190635,ЭЛИКВИС 5МГ. №60 ТАБ. П/П/О /ПФАЙЗЕР/БРИСТОЛ-М...,ГРАНД КАПИТАЛ СМОЛЕНСК ООО ФК,Пфайзер,1.00,2320.99,2563.00,76.00,9.00,200010018491.00,29,1,1.00
3,2022-08-01,09:00:40,1275,13002561,13,22589,Розничная реализация,18,276370,АРБИДОЛ МАКСИМУМ 200МГ. №10 КАПС. /ОТИСИФАРМ/Ф...,Пульс,ОТИСИФАРМ ПАО,1.00,445.39,541.00,0.00,NaN,NaN,29,1,1.00
4,2022-08-01,09:04:05,1276,13002561,13,22589,Розничная реализация,15,2303,"ЭНАМ 2,5МГ. №20 ТАБ. /Д-Р РЕДДИ/",Протек,Д-р Редди с Лабораторис Лтд / Dr.REDDY's,1.00,18.04,22.00,1.00,9.00,200010000734.00,29,1,5.00


In [ ]:
df = df.groupby(['DR_Dat', 'DR_NDrugs']).agg({'DR_Kol': sum})
df

DR_Kol
DR_Dat     DR_NDrugs                                                 
2022-08-01 911-ВЕНОЛГОН ГЕЛЬ Д/НОГ ПРИ ТЯЖЕСТИ,БОЛИ,ОТЕКАХ...    1.00
           L-ТИРОКСИН 100МКГ. №100 ТАБ. /БЕРЛИН ХЕМИ/            1.00
           L-ТИРОКСИН 50МКГ. №50 ТАБ. /БЕРЛИН ХЕМИ/              1.00
           L-ТИРОКСИН 75МКГ. №100 ТАБ. /БЕРЛИН ХЕМИ/             2.00
           NF ВАТА ХИРУРГ. СТЕР. 100Г. /НЬЮФАРМ/                 1.00
...                                                               ...
2022-08-05 ЭВО ПАНТЕНОЛ ПОМАДА ГИГИЕН. 2,8Г. [EVO] /АВАНТА/      1.00
           ЭМОКСИПИН 10МГ/МЛ. 5МЛ. №1 ГЛ.КАПЛИ ФЛ. КРЫШ/КАП.     2.00
           ЭСВИЦИН СР-ВО П/ОБЛЫСЕНИЯ ЛОСЬОН-ТОНИК 250МЛ. Ф...    1.00
           ЭТОРИАКС 90МГ. №7 ТАБ. П/П/О                          1.00
           ЮНИДОКС СОЛЮТАБ 100МГ. №20 ТАБ.ДИСПЕРГ. /АСТЕЛЛАС/    1.00

[1089 rows x 1 columns]

In [ ]:
df1 = df.reset_index().groupby('DR_NDrugs').agg({'DR_Dat': 'count'}).reset_index()
xyz_names = list(df1[df1['DR_Dat'] > 1]['DR_NDrugs'])
xyz_names

['L-ТИРОКСИН 50МКГ. №50 ТАБ. /БЕРЛИН ХЕМИ/',
 'L-ТИРОКСИН 75МКГ. №100 ТАБ. /БЕРЛИН ХЕМИ/',
 'АЗАРГА 10МГ/МЛ.+5МГ/МЛ. 5МЛ. №1 ГЛ.КАПЛИ ФЛ./КАП.',
 'АМОКСИЦИЛЛИН 500МГ. №16 КАПС. /ХЕМОФАРМ/',
 'АРБИДОЛ МАКСИМУМ 200МГ. №10 КАПС. /ОТИСИФАРМ/ФАРМСТАНДАРТ/',
 'АСЕПТИКА САЛФЕТКА СПИРТОВАЯ 60Х100 №20',
 'АСКОРБИНКА ЛУНТИК ВИТ.С КЛУБНИКА №10 ТАБ. (30Г.) /ФАРМ-ПРО/ТИГОДА-ФАРМ/',
 'АСКОРБИНКА ЛУНТИК ВИТ.С №10 ТАБ. (30Г.) /ФАРМ-ПРО/ТИГОДА-ФАРМ/',
 'АСКОРБИНОВАЯ К-ТА 25МГ. ВИШНЯ №10 ТАБ. КРУТКА САХ. /АСКОПРОМ/',
 'АСКОРБИНОВАЯ К-ТА 25МГ. ЯБЛОКО №10 ТАБ. КРУТКА САХ. /АСКОПРОМ/',
 'АСПАРКАМ №60 ТАБ. /ФАРМАПОЛ-ВОЛГА/',
 'АСПЕРА СУПЕРЧИСТОТЕЛ 3МЛ. ФЛ.',
 'АТЕНОЛОЛ РЕНЕВАЛ 50МГ. №30 ТАБ. /ОБНОВЛЕНИЕ/',
 'АТОПИК КРЕМ-СТИК УСПОКАИВАЮЩИЙ Д/ДЕТ. 0+ 4,9МЛ. ПЕНАЛ [ATOPIC]',
 'АТОРВАСТАТИН-СЗ 20МГ. №60 ТАБ. П/П/О /СЕВЕРНАЯ ЗВЕЗДА/',
 'АФОБАЗОЛ 10МГ. №60 ТАБ.',
 'АЦЕТИЛСАЛИЦИЛОВАЯ К-ТА 500МГ. №20 ТАБ. /ФАРМСТАНДАРТ/',
 'АЦИКЛОВИР-БЕЛУПО 5% 10Г. №1 КРЕМ Д/НАРУЖ.ПРИМ. ТУБА /БЕЛУПО/',
 'БЕТАГИСТИН-СЗ 16МГ. №60 ТАБ

In [ ]:
df = df.reset_index()
df = df[df['DR_NDrugs'].isin(xyz_names)]
df = df.groupby('DR_NDrugs').apply(lambda x: x.std()/x.mean())
df

C:\Users\User\AppData\Local\Temp\ipykernel_9316\4270249517.py:3: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df = df.groupby('DR_NDrugs').apply(lambda x: x.std()/x.mean())


,DR_Kol
DR_NDrugs,
L-ТИРОКСИН 50МКГ. №50 ТАБ. /БЕРЛИН ХЕМИ/,NaN
L-ТИРОКСИН 75МКГ. №100 ТАБ. /БЕРЛИН ХЕМИ/,NaN
АЗАРГА 10МГ/МЛ.+5МГ/МЛ. 5МЛ. №1 ГЛ.КАПЛИ ФЛ./КАП.,NaN
АМОКСИЦИЛЛИН 500МГ. №16 КАПС. /ХЕМОФАРМ/,NaN
АРБИДОЛ МАКСИМУМ 200МГ. №10 КАПС. /ОТИСИФАРМ/ФАРМСТАНДАРТ/,NaN
...,...
"ЭВО ПАНТЕНОЛ ПОМАДА ГИГИЕН. 2,8Г. [EVO] /АВАНТА/",NaN
ЭЛИКВИС 5МГ. №60 ТАБ. П/П/О /ПФАЙЗЕР/БРИСТОЛ-МАЙЕРС/,NaN
ЭМОКСИПИН 10МГ/МЛ. 5МЛ. №1 ГЛ.КАПЛИ ФЛ. КРЫШ/КАП.,NaN


In [ ]:
df

,level_0,index,DR_Kol,XYZ
DR_NDrugs,,,,
L-ТИРОКСИН 50МКГ. №50 ТАБ. /БЕРЛИН ХЕМИ/,1.41,1.41,0.00,X
L-ТИРОКСИН 75МКГ. №100 ТАБ. /БЕРЛИН ХЕМИ/,1.40,1.40,0.47,Z
АЗАРГА 10МГ/МЛ.+5МГ/МЛ. 5МЛ. №1 ГЛ.КАПЛИ ФЛ./КАП.,1.36,1.36,0.00,X
АМОКСИЦИЛЛИН 500МГ. №16 КАПС. /ХЕМОФАРМ/,0.60,0.60,0.47,Z
АРБИДОЛ МАКСИМУМ 200МГ. №10 КАПС. /ОТИСИФАРМ/ФАРМСТАНДАРТ/,1.13,1.13,0.43,Z
...,...,...,...,...
"ЭВО ПАНТЕНОЛ ПОМАДА ГИГИЕН. 2,8Г. [EVO] /АВАНТА/",0.15,0.15,0.47,Z
ЭЛИКВИС 5МГ. №60 ТАБ. П/П/О /ПФАЙЗЕР/БРИСТОЛ-МАЙЕРС/,0.42,0.42,0.00,X
ЭМОКСИПИН 10МГ/МЛ. 5МЛ. №1 ГЛ.КАПЛИ ФЛ. КРЫШ/КАП.,0.31,0.31,0.47,Z


In [ ]:
df['XYZ'] = np.where(df['DR_Kol'] < 0.1, 'X', np.where(df['DR_Kol'] < 0.25, 'Y', 'Z'))

In [ ]:
df

,level_0,index,DR_Kol,XYZ
DR_NDrugs,,,,
L-ТИРОКСИН 50МКГ. №50 ТАБ. /БЕРЛИН ХЕМИ/,1.41,1.41,0.00,X
L-ТИРОКСИН 75МКГ. №100 ТАБ. /БЕРЛИН ХЕМИ/,1.40,1.40,0.47,Z
АЗАРГА 10МГ/МЛ.+5МГ/МЛ. 5МЛ. №1 ГЛ.КАПЛИ ФЛ./КАП.,1.36,1.36,0.00,X
АМОКСИЦИЛЛИН 500МГ. №16 КАПС. /ХЕМОФАРМ/,0.60,0.60,0.47,Z
АРБИДОЛ МАКСИМУМ 200МГ. №10 КАПС. /ОТИСИФАРМ/ФАРМСТАНДАРТ/,1.13,1.13,0.43,Z
...,...,...,...,...
"ЭВО ПАНТЕНОЛ ПОМАДА ГИГИЕН. 2,8Г. [EVO] /АВАНТА/",0.15,0.15,0.47,Z
ЭЛИКВИС 5МГ. №60 ТАБ. П/П/О /ПФАЙЗЕР/БРИСТОЛ-МАЙЕРС/,0.42,0.42,0.00,X
ЭМОКСИПИН 10МГ/МЛ. 5МЛ. №1 ГЛ.КАПЛИ ФЛ. КРЫШ/КАП.,0.31,0.31,0.47,Z


In [ ]:
a = {'b': [1]}
a = pd.DataFrame(a)
a

,b
0,1


In [ ]:
a['b'].std()

0.7071067811865476

In [ ]:
np.std(a['b'])

0.0